In [1]:
pip install --upgrade pymupdf

     |████████████████████████████████| 4.3 MB 3.2 MB/s eta 0:00:01
     |████████████████████████████████| 30.6 MB 5.3 MB/s eta 0:00:01
  Attempting uninstall: PyMuPDFb
    Found existing installation: PyMuPDFb 1.23.5
    Uninstalling PyMuPDFb-1.23.5:
      Successfully uninstalled PyMuPDFb-1.23.5
  Attempting uninstall: pymupdf
    Found existing installation: PyMuPDF 1.23.5
    Uninstalling PyMuPDF-1.23.5:
      Successfully uninstalled PyMuPDF-1.23.5
Note: you may need to restart the kernel to use updated packages.


In [2]:
import fitz

In [18]:
doc = fitz.open("../resources/testpdf/marx.pdf", filetype="pdf")

In [19]:
doc.page_count

1

In [20]:

import json
#f = open('test.html', 'w') 

pages_json = {}

pages = doc.pages()
for page in pages:
    # https://pymupdf.readthedocs.io/en/latest/page.html#Page.get_text
    page_cropbox = page.cropbox
    page_info = page.get_text("rawjson")
    page_json = json.loads(page_info)
    pages_json[page.number] = page_json

# blocks of page 1
print(pages_json[0]['blocks'])

[{'number': 0, 'type': 0, 'bbox': [119.05500030517578, 109.42204284667969, 544.47900390625, 140.47802734375], 'lines': [{'spans': [{'size': 12.0, 'flags': 20, 'font': 'UniversLT-Bold', 'color': 0, 'ascender': 0.9380000233650208, 'descender': -0.25, 'chars': [{'origin': [119.05500030517578, 120.67803955078125], 'bbox': [119.05500030517578, 109.42204284667969, 127.71900177001953, 123.67803955078125], 'c': 'A'}, {'origin': [127.71900177001953, 120.67803955078125], 'bbox': [127.71900177001953, 109.42204284667969, 134.39100646972656, 123.67803955078125], 'c': 'a'}, {'origin': [134.39100646972656, 120.67803955078125], 'bbox': [134.39100646972656, 109.42204284667969, 141.72300720214844, 123.67803955078125], 'c': 'n'}, {'origin': [141.48300170898438, 120.67803955078125], 'bbox': [141.48300170898438, 109.42204284667969, 148.15499877929688, 123.67803955078125], 'c': 'v'}, {'origin': [148.15499877929688, 120.67803955078125], 'bbox': [148.15499877929688, 109.42204284667969, 152.822998046875, 123.6

In [21]:
"""
search for 'PDF' text on a page and do some tests
"""
doc.save("test1_test.pdf", garbage=3, deflate=True)
doc2 = fitz.open("test1_test.pdf", filetype="pdf")

pages = doc2.pages()
for page in pages:
    
    # Dit is de methode van tekst selecteren, of dat nou een text input field is 
    # of het een manier is om text de selecteren in een GUI
    res = page.search_for('pdf')
    
    # get Rect of text, including the bounding box of text.
    print(res)
    
    # test to extract text using rect
    res2 = page.get_text(clip=res[0])
    print(res2)
    
    # Make Quad of Rect
    res_quad = res[0].quad
    print(res_quad)
    
    # add Redact annotation to select text to be redacted
    redaction = page.add_redact_annot(res_quad,  cross_out=True)
    
    doc2.save("test1_annot.pdf", garbage=4, deflate=True)
    
    page.apply_redactions()
    point = (res[0][0], res[0][3])
    page.add_rect_annot(res[0])
    page.insert_text(point=point , text="pdf", fontname=page.get_fonts()[0][4], color=(0, 0, 0), fontsize=11)


doc2.save("test1_res.pdf", garbage=4, clean=True, deflate=True)
doc2 = fitz.open("test1_res.pdf", filetype="pdf")

[]


IndexError: list index out of range

In [22]:
doc_object = {}
xreflen = doc2.xref_length()  # length of objects table
print(xreflen)
with open('marx.txt', 'w') as f:
    for xref in range(1, xreflen):  # skip item 0!
        f.write(doc2.xref_object(xref, compressed=False))
        
        keys = doc2.xref_get_keys(xref)
        object_keys = {}
        for key in keys:
            value = doc2.xref_get_key(xref, key)
            object_keys[key] = value
        if stream := doc2.xref_stream(xref):
            f.write(stream.decode("latin-1"))
            object_keys["stream"] = stream
                
            
        doc_object["object%i" % (xref)] = object_keys
        

59
